In [1]:
from models import *
from utils.utils import *
import numpy as np
from copy import deepcopy
from test import test
from terminaltables import AsciiTable
import time
from utils.prune_utils import *
import argparse

#0.419     0.473

In [2]:
parser = argparse.ArgumentParser()
parser.add_argument('--cfg', type=str, default='cfg/dense_yolov3_4.cfg', help='cfg file path')
parser.add_argument('--data', type=str, default='data/visdrone.data', help='*.data file path')
parser.add_argument('--weights', type=str, default='weights/best.pt', help='sparse model weights')
parser.add_argument('--global_percent', type=float, default=0.8, help='global channel prune percent')
parser.add_argument('--layer_keep', type=float, default=0.01, help='channel keep percent per layer')
parser.add_argument('--img_size', type=int, default=800, help='inference size (pixels)')
opt = parser.parse_known_args()[0]
opt.cfg = check_file(opt.cfg)  # check file
opt.data = check_file(opt.data)

In [3]:
img_size = opt.img_size
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = Darknet(opt.cfg, (img_size, img_size)).to(device)

if opt.weights.endswith(".pt"):
    model.load_state_dict(torch.load(opt.weights, map_location=device)['model'])
else:
    _ = load_darknet_weights(model, opt.weights)
print('\nloaded weights from ',opt.weights)

eval_model = lambda model:test(model = model, cfg=opt.cfg, data=opt.data,batch_size=64, imgsz=img_size,is_training = False)
obtain_num_parameters = lambda model:sum([param.nelement() for param in model.parameters()])

print("\nlet's test the original model first:")
with torch.no_grad():
    origin_model_metric = eval_model(model)
origin_nparameters = obtain_num_parameters(model)

Model Summary: 254 layers, 6.19432e+07 parameters, 6.19432e+07 gradients


Caching labels /home/tione/notebook/seu/VisDrone2019-DET-val/labels.txt (548 found, 0 missing, 0 empty, 0 duplicate, for 548 images): 100%|██████████| 548/548 [00:00<00:00, 3346.30it/s]


loaded weights from  weights/best.pt

let's test the original model first:



               Class    Images   Targets         P         R   mAP@0.5        F1:   0%|          | 0/9 [00:00<?, ?it/s]/home/tione/notebook/seu/yolov3-master/utils/utils.py:512: UserWarning: This overload of nonzero is deprecated:
	nonzero()
Consider using one of the following signatures instead:
	nonzero(*, bool as_tuple) (Triggered internally at  /opt/conda/conda-bld/pytorch_1595629411241/work/torch/csrc/utils/python_arg_parser.cpp:766.)
  i, j = (x[:, 5:] > conf_thres).nonzero().t()
               Class    Images   Targets         P         R   mAP@0.5        F1: 100%|██████████| 9/9 [01:48<00:00, 12.05s/it]


                 all       548  3.88e+04     0.398     0.484     0.422     0.435


In [4]:
CBL_idx, Conv_idx, prune_idx, _, _= parse_module_defs2(model.module_defs)
bn_weights = gather_bn_weights(model.module_list, prune_idx)

sorted_bn = torch.sort(bn_weights)[0]
sorted_bn, sorted_index = torch.sort(bn_weights)
thresh_index = int(len(bn_weights) * opt.global_percent)
thresh = sorted_bn[thresh_index].cuda()

print(f'Global Threshold should be less than {thresh:.4f}.')

Global Threshold should be less than 0.0000.


In [5]:
def obtain_filters_mask(model, thre, CBL_idx, prune_idx):

    pruned = 0
    total = 0
    num_filters = []
    filters_mask = []
    for idx in CBL_idx:
        bn_module = model.module_list[idx][1]
        if idx in prune_idx:

            weight_copy = bn_module.weight.data.abs().clone()

            channels = weight_copy.shape[0] #
            min_channel_num = int(channels * opt.layer_keep) if int(channels * opt.layer_keep) > 0 else 1
            mask = weight_copy.gt(thresh).float()

            if int(torch.sum(mask)) < min_channel_num: 
                _, sorted_index_weights = torch.sort(weight_copy,descending=True)
                mask[sorted_index_weights[:min_channel_num]]=1. 
            remain = int(mask.sum())
            pruned = pruned + mask.shape[0] - remain

            print(f'layer index: {idx:>3d} \t total channel: {mask.shape[0]:>4d} \t '
                    f'remaining channel: {remain:>4d}')
        else:
            mask = torch.ones(bn_module.weight.data.shape)
            remain = mask.shape[0]

        total += mask.shape[0]
        num_filters.append(remain)
        filters_mask.append(mask.clone())

    prune_ratio = pruned / total
    print(f'Prune channels: {pruned}\tPrune ratio: {prune_ratio:.3f}')

    return num_filters, filters_mask

num_filters, filters_mask = obtain_filters_mask(model, thresh, CBL_idx, prune_idx)
CBLidx2mask = {idx: mask for idx, mask in zip(CBL_idx, filters_mask)}
CBLidx2filters = {idx: filters for idx, filters in zip(CBL_idx, num_filters)}

layer index:   0 	 total channel:   32 	 remaining channel:   29
layer index:   1 	 total channel:   64 	 remaining channel:   55
layer index:   2 	 total channel:   32 	 remaining channel:   28
layer index:   3 	 total channel:   64 	 remaining channel:   39
layer index:   5 	 total channel:  128 	 remaining channel:  119
layer index:   6 	 total channel:   64 	 remaining channel:   32
layer index:   7 	 total channel:  128 	 remaining channel:   77
layer index:   9 	 total channel:   64 	 remaining channel:   52
layer index:  10 	 total channel:  128 	 remaining channel:   84
layer index:  12 	 total channel:  256 	 remaining channel:  192
layer index:  13 	 total channel:  128 	 remaining channel:   38
layer index:  14 	 total channel:  256 	 remaining channel:   14
layer index:  16 	 total channel:  128 	 remaining channel:    6
layer index:  17 	 total channel:  256 	 remaining channel:   19
layer index:  19 	 total channel:  128 	 remaining channel:    5
layer index:  20 	 total 

In [6]:
for i in model.module_defs:
    if i['type'] == 'shortcut':
        i['is_access'] = False

print('merge the mask of layers connected to shortcut!')
merge_mask(model, CBLidx2mask, CBLidx2filters)

merge the mask of layers connected to shortcut!


In [7]:
for i in CBLidx2mask:
    CBLidx2mask[i] = CBLidx2mask[i].clone().cpu().numpy()

pruned_model = prune_model_keep_size2(model, prune_idx, CBL_idx, CBLidx2mask)
print("\nnow prune the model but keep size,(actually add offset of BN beta to following layers), let's see how the mAP goes")



now prune the model but keep size,(actually add offset of BN beta to following layers), let's see how the mAP goes


In [8]:
# def prune_and_eval(model, CBL_idx, CBLidx2mask):
#     model_copy = deepcopy(model)

#     for idx in CBL_idx:
#         bn_module = model_copy.module_list[idx][1]
#         mask = CBLidx2mask[idx].cuda()
#         bn_module.weight.data.mul_(mask)

#     with torch.no_grad():
#         mAP = eval_model(model_copy)[0][2]

#     print(f'mask the gamma as zero, mAP of the model is {mAP:.4f}')


# prune_and_eval(model, CBL_idx, CBLidx2mask)

In [9]:
for i in model.module_defs:
    if i['type'] == 'shortcut':
        i.pop('is_access')

compact_module_defs = deepcopy(model.module_defs)
for idx in CBL_idx:
    assert compact_module_defs[idx]['type'] == 'convolutional'
    compact_module_defs[idx]['filters'] = str(CBLidx2filters[idx])


compact_model = Darknet([model.hyperparams.copy()] + compact_module_defs, (img_size, img_size)).to(device)
compact_nparameters = obtain_num_parameters(compact_model)

init_weights_from_loose_model(compact_model, pruned_model, CBL_idx, Conv_idx, CBLidx2mask)


random_input = torch.rand((1, 3, img_size, img_size)).to(device)

Model Summary: 254 layers, 4.9728e+06 parameters, 4.9728e+06 gradients


In [10]:
def obtain_avg_forward_time(input, model, repeat=200):

    model.eval()
    start = time.time()
    with torch.no_grad():
        for i in range(repeat):
            output = model(input)
    avg_infer_time = (time.time() - start) / repeat

    return avg_infer_time, output

print('testing inference time...')
pruned_forward_time, pruned_output = obtain_avg_forward_time(random_input, pruned_model)
compact_forward_time, compact_output = obtain_avg_forward_time(random_input, compact_model)

print('testing the final model...')
with torch.no_grad():
    compact_model_metric = eval_model(compact_model)
    
metric_table = [

    ["Metric", "Before", "After"],
    ["mAP", f'{origin_model_metric[0][2]:.6f}', f'{compact_model_metric[0][2]:.6f}'],
    ["Parameters", f"{origin_nparameters}", f"{compact_nparameters}"],
    ["Inference", f'{pruned_forward_time:.4f}', f'{compact_forward_time:.4f}']
]
print(AsciiTable(metric_table).table)



pruned_cfg_name = opt.cfg.replace('/', f'/prune_{opt.global_percent}_keep_{opt.layer_keep}_')
pruned_cfg_file = write_cfg(pruned_cfg_name, [model.hyperparams.copy()] + compact_module_defs)
print(f'Config file has been saved: {pruned_cfg_file}')

compact_model_name = opt.weights.replace('/', f'/prune_{opt.global_percent}_keep_{opt.layer_keep}_')
if compact_model_name.endswith('.pt'):
    compact_model_name = compact_model_name.replace('.pt', '.weights')
save_weights(compact_model, path=compact_model_name)
print(f'Compact model has been saved: {compact_model_name}')

testing inference time...


Caching labels /home/tione/notebook/seu/VisDrone2019-DET-val/labels.txt (548 found, 0 missing, 0 empty, 0 duplicate, for 548 images): 100%|██████████| 548/548 [00:00<00:00, 3342.33it/s]
               Class    Images   Targets         P         R   mAP@0.5        F1:   0%|          | 0/9 [00:00<?, ?it/s]

testing the final model...


               Class    Images   Targets         P         R   mAP@0.5        F1: 100%|██████████| 9/9 [01:45<00:00, 11.74s/it]


                 all       548  3.88e+04     0.398     0.484     0.421     0.435
+------------+----------+----------+
| Metric     | Before   | After    |
+------------+----------+----------+
| mAP        | 0.421684 | 0.420999 |
| Parameters | 61943156 | 4972797  |
| Inference  | 0.0230   | 0.0125   |
+------------+----------+----------+
Config file has been saved: cfg/prune_0.8_keep_0.01_dense_yolov3_4.cfg
Compact model has been saved: weights/prune_0.8_keep_0.01_best.weights


In [11]:
img_size = [600,800]
img_size.extend([img_size[-1]] * (3 - len(img_size)))
img_size

[600, 800, 800]

In [19]:
pruned_model.module_list

ModuleList(
  (0): Sequential(
    (Conv2d): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (BatchNorm2d): BatchNorm2d(32, eps=0.0001, momentum=0.03, affine=True, track_running_stats=True)
    (activation): LeakyReLU(negative_slope=0.1, inplace=True)
  )
  (1): Sequential(
    (Conv2d): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (BatchNorm2d): BatchNorm2d(64, eps=0.0001, momentum=0.03, affine=True, track_running_stats=True)
    (activation): LeakyReLU(negative_slope=0.1, inplace=True)
  )
  (2): Sequential(
    (Conv2d): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (BatchNorm2d): BatchNorm2d(32, eps=0.0001, momentum=0.03, affine=True, track_running_stats=True)
    (activation): LeakyReLU(negative_slope=0.1, inplace=True)
  )
  (3): Sequential(
    (Conv2d): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (BatchNorm2d): BatchNorm2d(64, eps=0.0001, momentum=0.03

In [17]:
pruned_model.module_defs

[{'type': 'convolutional',
  'batch_normalize': 1,
  'filters': 32,
  'size': 3,
  'stride': 1,
  'pad': 1,
  'activation': 'leaky'},
 {'type': 'convolutional',
  'batch_normalize': 1,
  'filters': 64,
  'size': 3,
  'stride': 2,
  'pad': 1,
  'activation': 'leaky'},
 {'type': 'convolutional',
  'batch_normalize': 1,
  'filters': 32,
  'size': 1,
  'stride': 1,
  'pad': 1,
  'activation': 'leaky'},
 {'type': 'convolutional',
  'batch_normalize': 1,
  'filters': 64,
  'size': 3,
  'stride': 1,
  'pad': 1,
  'activation': 'leaky'},
 {'type': 'shortcut', 'from': [-3], 'activation': 'linear', 'is_access': True},
 {'type': 'convolutional',
  'batch_normalize': 1,
  'filters': 128,
  'size': 3,
  'stride': 2,
  'pad': 1,
  'activation': 'leaky'},
 {'type': 'convolutional',
  'batch_normalize': 1,
  'filters': 64,
  'size': 1,
  'stride': 1,
  'pad': 1,
  'activation': 'leaky'},
 {'type': 'convolutional',
  'batch_normalize': 1,
  'filters': 128,
  'size': 3,
  'stride': 1,
  'pad': 1,
  'act

In [23]:
a=np.array([[1,2,3],[4,5,6]])
b=np.array([[11,21,31],[7,8,9]])
np.concatenate([a,b])

array([[ 1,  2,  3],
       [ 4,  5,  6],
       [11, 21, 31],
       [ 7,  8,  9]])

In [40]:
i =12

if i ==1:
    print('1')
elif i ==2:
    print('2')
else:
    print('3')

3


In [31]:
CBLidx2mask[0]

array([          1,           1,           1,           1,           1,           1,           1,           1,           1,           1,           1,           1,           1,           1,           1,           1,           1,           1,           1,           1,           1,           1,           1,           1,
                 1,           1,           1,           1,           1,           0,           1,           1], dtype=float32)

In [33]:
np.concatenate([CBLidx2mask[0],CBLidx2mask[1],CBLidx2mask[5]])

array([          1,           1,           1,           1,           1,           1,           1,           1,           1,           1,           1,           1,           1,           1,           1,           1,           1,           1,           1,           1,           1,           1,           1,           1,
                 1,           1,           1,           1,           1,           0,           1,           1,           1,           0,           1,           1,           1,           1,           1,           1,           1,           0,           1,           1,           1,           1,           1,           1,
                 1,           1,           1,           1,           1,           1,           1,           1,           1,           1,           1,           1,           1,           1,           1,           1,           1,           1,           1,           1,           1,           1,           1,           1,
                 1,           1,           